# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb86cd59bb0>
├── 'a' --> tensor([[-0.1275, -0.0319, -1.6389],
│                   [-0.0040, -0.7521,  0.4777]])
└── 'x' --> <FastTreeValue 0x7fb86cd59ee0>
    └── 'c' --> tensor([[-0.2708, -1.0875, -0.0965,  0.1577],
                        [ 0.3538, -0.6505, -1.9733, -0.5614],
                        [-1.0685,  0.6300,  0.3295, -0.0028]])

In [4]:
t.a

tensor([[-0.1275, -0.0319, -1.6389],
        [-0.0040, -0.7521,  0.4777]])

In [5]:
%timeit t.a

61.5 ns ± 0.0855 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb86cd59bb0>
├── 'a' --> tensor([[-0.3278,  0.3624,  0.6026],
│                   [ 1.3623, -0.5885,  0.7675]])
└── 'x' --> <FastTreeValue 0x7fb86cd59ee0>
    └── 'c' --> tensor([[-0.2708, -1.0875, -0.0965,  0.1577],
                        [ 0.3538, -0.6505, -1.9733, -0.5614],
                        [-1.0685,  0.6300,  0.3295, -0.0028]])

In [7]:
%timeit t.a = new_value

63.5 ns ± 0.072 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1275, -0.0319, -1.6389],
               [-0.0040, -0.7521,  0.4777]]),
    x: Batch(
           c: tensor([[-0.2708, -1.0875, -0.0965,  0.1577],
                      [ 0.3538, -0.6505, -1.9733, -0.5614],
                      [-1.0685,  0.6300,  0.3295, -0.0028]]),
       ),
)

In [10]:
b.a

tensor([[-0.1275, -0.0319, -1.6389],
        [-0.0040, -0.7521,  0.4777]])

In [11]:
%timeit b.a

56.7 ns ± 0.0959 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2791,  0.1014, -1.2108],
               [-0.0763,  0.4632,  0.1254]]),
    x: Batch(
           c: tensor([[-0.2708, -1.0875, -0.0965,  0.1577],
                      [ 0.3538, -0.6505, -1.9733, -0.5614],
                      [-1.0685,  0.6300,  0.3295, -0.0028]]),
       ),
)

In [13]:
%timeit b.a = new_value

488 ns ± 0.135 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

841 ns ± 0.855 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 40.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 822 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

142 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb86cd59a30>
├── 'a' --> tensor([[[-0.1275, -0.0319, -1.6389],
│                    [-0.0040, -0.7521,  0.4777]],
│           
│                   [[-0.1275, -0.0319, -1.6389],
│                    [-0.0040, -0.7521,  0.4777]],
│           
│                   [[-0.1275, -0.0319, -1.6389],
│                    [-0.0040, -0.7521,  0.4777]],
│           
│                   [[-0.1275, -0.0319, -1.6389],
│                    [-0.0040, -0.7521,  0.4777]],
│           
│                   [[-0.1275, -0.0319, -1.6389],
│                    [-0.0040, -0.7521,  0.4777]],
│           
│                   [[-0.1275, -0.0319, -1.6389],
│                    [-0.0040, -0.7521,  0.4777]],
│           
│                   [[-0.1275, -0.0319, -1.6389],
│                    [-0.0040, -0.7521,  0.4777]],
│           
│                   [[-0.1275, -0.0319, -1.6389],
│                    [-0.0040, -0.7521,  0.4777]]])
└── 'x' --> <FastTreeValue 0x7fb84452b460>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 40.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb7aa1666d0>
├── 'a' --> tensor([[-0.1275, -0.0319, -1.6389],
│                   [-0.0040, -0.7521,  0.4777],
│                   [-0.1275, -0.0319, -1.6389],
│                   [-0.0040, -0.7521,  0.4777],
│                   [-0.1275, -0.0319, -1.6389],
│                   [-0.0040, -0.7521,  0.4777],
│                   [-0.1275, -0.0319, -1.6389],
│                   [-0.0040, -0.7521,  0.4777],
│                   [-0.1275, -0.0319, -1.6389],
│                   [-0.0040, -0.7521,  0.4777],
│                   [-0.1275, -0.0319, -1.6389],
│                   [-0.0040, -0.7521,  0.4777],
│                   [-0.1275, -0.0319, -1.6389],
│                   [-0.0040, -0.7521,  0.4777],
│                   [-0.1275, -0.0319, -1.6389],
│                   [-0.0040, -0.7521,  0.4777]])
└── 'x' --> <FastTreeValue 0x7fb86cd8b7f0>
    └── 'c' --> tensor([[-0.2708, -1.0875, -0.0965,  0.1577],
                        [ 0.3538, -0.6505, -1.9733, -0.5614],
                 

In [23]:
%timeit t_cat(trees)

29.9 µs ± 63.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

69.3 µs ± 43.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.2708, -1.0875, -0.0965,  0.1577],
                       [ 0.3538, -0.6505, -1.9733, -0.5614],
                       [-1.0685,  0.6300,  0.3295, -0.0028]],
              
                      [[-0.2708, -1.0875, -0.0965,  0.1577],
                       [ 0.3538, -0.6505, -1.9733, -0.5614],
                       [-1.0685,  0.6300,  0.3295, -0.0028]],
              
                      [[-0.2708, -1.0875, -0.0965,  0.1577],
                       [ 0.3538, -0.6505, -1.9733, -0.5614],
                       [-1.0685,  0.6300,  0.3295, -0.0028]],
              
                      [[-0.2708, -1.0875, -0.0965,  0.1577],
                       [ 0.3538, -0.6505, -1.9733, -0.5614],
                       [-1.0685,  0.6300,  0.3295, -0.0028]],
              
                      [[-0.2708, -1.0875, -0.0965,  0.1577],
                       [ 0.3538, -0.6505, -1.9733, -0.5614],
                       [-1.0685,  0.6300,  0.3295, -0.0028]],

In [26]:
%timeit Batch.stack(batches)

79.7 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.2708, -1.0875, -0.0965,  0.1577],
                      [ 0.3538, -0.6505, -1.9733, -0.5614],
                      [-1.0685,  0.6300,  0.3295, -0.0028],
                      [-0.2708, -1.0875, -0.0965,  0.1577],
                      [ 0.3538, -0.6505, -1.9733, -0.5614],
                      [-1.0685,  0.6300,  0.3295, -0.0028],
                      [-0.2708, -1.0875, -0.0965,  0.1577],
                      [ 0.3538, -0.6505, -1.9733, -0.5614],
                      [-1.0685,  0.6300,  0.3295, -0.0028],
                      [-0.2708, -1.0875, -0.0965,  0.1577],
                      [ 0.3538, -0.6505, -1.9733, -0.5614],
                      [-1.0685,  0.6300,  0.3295, -0.0028],
                      [-0.2708, -1.0875, -0.0965,  0.1577],
                      [ 0.3538, -0.6505, -1.9733, -0.5614],
                      [-1.0685,  0.6300,  0.3295, -0.0028],
                      [-0.2708, -1.0875, -0.0965,  0.1577],
                   

In [28]:
%timeit Batch.cat(batches)

148 µs ± 276 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

338 µs ± 3.67 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
